<a href="https://colab.research.google.com/github/PaulYYLin/MINST_KaggleComp/blob/main/MINST_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Kaggle Connection

In [ ]:
!pip install kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json



In [ ]:
!kaggle competitions list

In [ ]:
!kaggle competitions download "digit-recognizer"

 52% 8.00M/15.3M [00:00<00:00, 80.9MB/s]
100% 15.3M/15.3M [00:00<00:00, 48.0MB/s]


In [ ]:
!unzip digit-recognizer.zip

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd

### Data Description
Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

In [ ]:
train = pd.read_csv('train.csv')

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

In [ ]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Preparation

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
x = train.iloc[:,1:]
y = train.iloc[:,0]


data = tf.data.Dataset.from_tensor_slices((x,y))


In [ ]:
print(x.shape, y.shape)

(42000, 784) (42000,)


In [ ]:
# Data casting
def feature_scale(x,y):
  x = tf.cast(x, dtype=tf.float32)/255 # Cast to 256 dimension
  y = tf.cast(y, dtype = tf.int32)
  return x,y

# Label to one-hot
y = tf.keras.utils.to_categorical(
    y, num_classes=10)

# map: 將每一筆資料帶入 function (cast), shuffle: 重混資料, 設定batch_size
data = data.map(feature_scale).shuffle(10000).batch(128)

In [ ]:
data_iter = iter(data)
sample = next(data_iter)

print(sample[0].shape, sample[1].shape)

(128, 784) (128,)


## Model Compile

In [ ]:
#定義Model

model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.build(input_shape=[1,28*28])

tf.keras.losses https://www.tensorflow.org/api_docs/python/tf/keras/losses

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['crossentropy'],
)
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (1, 256)                  200960    
                                                                 
 dense_55 (Dense)            (1, 128)                  32896     
                                                                 
 dense_56 (Dense)            (1, 64)                   8256      
                                                                 
 dense_57 (Dense)            (1, 32)                   2080      
                                                                 
 dense_58 (Dense)            (1, 16)                   528       
                                                                 
 dense_59 (Dense)            (1, 10)                   170       
                                                                 
Total params: 244,890
Trainable params: 244,890
Non-tr

## Model Fit / Evaluate

In [ ]:
history = model.fit(x.values, y, epochs=100, validation_split = 0.1)

In [ ]:
model.save('Mymodel')

## Model Predict

In [ ]:
model.load('Mymodel')

In [ ]:
test = pd.read_csv('test.csv')
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_test = tf.data.Dataset.from_tensor_slices(test)

In [ ]:
t_hat = model.predict(test, batch_size = 1000)

28/28 [==============================] - 1s 16ms/step


In [ ]:
final_hat = [tf.math.argmax(prob).numpy() for prob in t_hat]
final_hat[:10]

[2, 0, 9, 4, 3, 7, 0, 3, 0, 3]

In [ ]:
result = pd.read_csv('sample_submission.csv')
result['Label'] = final_hat
result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,4
4,5,3


In [ ]:
result.to_csv('h1_hat.csv', index=False)

Model H1 predict result: Acc 0.9751